In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## **TASK 1**

In [283]:
dataset = pd.read_csv("../Downloads/archive/tested.csv")
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [284]:
X = dataset.iloc[:, [4,2,5,6,7]].values
Y = dataset.iloc[:, 1].values

In [285]:
pd.DataFrame(X).head()

,0,1,2,3,4
0,male,3,34.5,0,0
1,female,3,47.0,1,0
2,male,2,62.0,0,0
3,male,3,27.0,0,0
4,female,3,22.0,1,1


In [286]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
enc = le.fit_transform(X[:,0])
X[:,0] = enc
pd.DataFrame(X).head()

,0,1,2,3,4
0,1,3,34.5,0,0
1,0,3,47.0,1,0
2,1,2,62.0,0,0
3,1,3,27.0,0,0
4,0,3,22.0,1,1


In [287]:
pd.DataFrame(X).iloc[:, 1].value_counts()

3    218
1    107
2     93
Name: 1, dtype: int64

In [288]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
pd.DataFrame(X).head()

,0,1,2,3,4,5
0,0.0,1.0,1,34.5,0,0
1,0.0,1.0,0,47.0,1,0
2,1.0,0.0,1,62.0,0,0
3,0.0,1.0,1,27.0,0,0
4,0.0,1.0,0,22.0,1,1


In [289]:
pd.DataFrame(X).isnull().value_counts()

0      1      2      3      4      5    
False  False  False  False  False  False    332
                     True   False  False     86
dtype: int64

In [290]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X)
X = imputer.transform(X)
pd.DataFrame(X).head()

,0,1,2,3,4,5
0,0.0,1.0,1.0,34.5,0.0,0.0
1,0.0,1.0,0.0,47.0,1.0,0.0
2,1.0,0.0,1.0,62.0,0.0,0.0
3,0.0,1.0,1.0,27.0,0.0,0.0
4,0.0,1.0,0.0,22.0,1.0,1.0


In [291]:
pd.DataFrame(X).isnull().value_counts()

0      1      2      3      4      5    
False  False  False  False  False  False    418
dtype: int64

In [292]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 3:4] = sc.fit_transform(X_train[:, 3:4])
X_test[:, 3:4] = sc.transform(X_test[:, 3:4])
pd.DataFrame(X_train).head()

,0,1,2,3,4,5
0,0.0,1.0,1.0,-0.012461,0.0,0.0
1,0.0,0.0,0.0,-0.033842,0.0,0.0
2,0.0,1.0,0.0,-0.012461,0.0,2.0
3,0.0,1.0,1.0,-0.012461,0.0,0.0
4,0.0,1.0,1.0,0.201474,0.0,0.0


In [293]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm = pd.DataFrame(cm, columns = ['Pred 0', 'Pred 1'], index = ['Actual 0', 'Actual 1'])
cm

,Pred 0,Pred 1
Actual 0,50,0
Actual 1,0,34


In [294]:
accuracy_score(y_test, y_pred)

1.0

## **TASK 2**

In [295]:
dataset = pd.read_csv("../Downloads/archive/IMDb_Movies_India.csv", encoding='latin1')
dataset.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [296]:
dataset.isna().sum()

Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

In [297]:
for c in dataset.columns:
    print ("---- %s ---" % c)
    print (dataset[c].value_counts())

---- Name ---
Anjaam                7
Mamta                 7
Zindagi               6
Musafir               6
Albela                5
                     ..
Hamen Bhi Jeene Do    1
Hamen Khelne Do       1
Hameshaa              1
Hamid                 1
Zulm-O-Sitam          1
Name: Name, Length: 13838, dtype: int64
---- Year ---
(2019)    410
(2021)    392
(2018)    381
(2017)    360
(2005)    346
         ... 
(1917)      1
(1913)      1
(1926)      1
(1914)      1
(1924)      1
Name: Year, Length: 102, dtype: int64
---- Duration ---
120 min    240
135 min    173
130 min    171
140 min    167
150 min    154
          ... 
201 min      1
206 min      1
198 min      1
218 min      1
250 min      1
Name: Duration, Length: 182, dtype: int64
---- Genre ---
Drama                         2780
Action                        1289
Thriller                       779
Romance                        708
Drama, Romance                 524
                              ... 
Action, Musical, War      

In [298]:
dataset = dataset.iloc[:, 1:]
dataset.head()

,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [299]:
dataset.dtypes

Year         object
Duration     object
Genre        object
Rating      float64
Votes        object
Director     object
Actor 1      object
Actor 2      object
Actor 3      object
dtype: object

In [300]:
dataset = dataset.dropna()
dataset.isnull().value_counts()

Year   Duration  Genre  Rating  Votes  Director  Actor 1  Actor 2  Actor 3
False  False     False  False   False  False     False    False    False      5659
dtype: int64

In [301]:
dataset['Year'] = dataset['Year'].str.replace(r'[()]', '', regex=True)
dataset['Duration'] = dataset['Duration'].str.replace(r' min', '', regex=True)
dataset.head()

,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
1,2019,109,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
3,2019,110,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
5,1997,147,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor
6,2005,142,"Drama, Romance, War",7.4,"1,086",Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma
8,2012,82,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia


In [302]:
X = dataset.drop('Rating', axis=1)
Y = dataset.iloc[:, 3].values
X.head()

,Year,Duration,Genre,Votes,Director,Actor 1,Actor 2,Actor 3
1,2019,109,Drama,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
3,2019,110,"Comedy, Romance",35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
5,1997,147,"Comedy, Drama, Musical",827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor
6,2005,142,"Drama, Romance, War","1,086",Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma
8,2012,82,"Horror, Mystery, Thriller",326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia


In [303]:
col_list = ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']
for col in col_list:
    dummies = X.loc[:, col].str.get_dummies(', ')
    X = X.drop(col, axis=1)
    X = pd.concat([X, dummies], axis=1)
X.head()

,Year,Duration,Votes,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,Zara Shah,Zareen Khan,Zarine Ali,Zayed Khan,Zeenat Aman,Zeeshan Khan,Zeishan Quadri,Zenobia Shroff,Zoya Hussain,Zulfi Sayed
1,2019,109,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019,110,35,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1997,147,827,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2005,142,"1,086",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2012,82,326,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [304]:
X.dtypes

Year              object
Duration          object
Votes             object
Action             int64
Adventure          int64
                   ...  
Zeeshan Khan       int64
Zeishan Quadri     int64
Zenobia Shroff     int64
Zoya Hussain       int64
Zulfi Sayed        int64
Length: 9293, dtype: object

In [305]:
col_list = ['Year', 'Duration', 'Votes']
for col in col_list:
    X[col] = X[col].str.replace(',', '').astype(float)
X.head()

,Year,Duration,Votes,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,Zara Shah,Zareen Khan,Zarine Ali,Zayed Khan,Zeenat Aman,Zeeshan Khan,Zeishan Quadri,Zenobia Shroff,Zoya Hussain,Zulfi Sayed
1,2019.0,109.0,8.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019.0,110.0,35.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1997.0,147.0,827.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2005.0,142.0,1086.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2012.0,82.0,326.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [306]:
Y = Y.reshape(Y.shape[0],1)
Y.shape

(5659, 1)

In [307]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, Y, test_size = 0.2)
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_train[:, :3] = sc_x.fit_transform(X_train[:, :3])
X_test[:, :3] = sc_x.transform(X_test[:, :3])
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train)
y_test = sc_y.transform(y_test)
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,9283,9284,9285,9286,9287,9288,9289,9290,9291,9292
0,-0.880333,0.799639,-0.186969,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.944626,0.128405,-0.190062,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.488043,-0.108501,-0.187450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.133736,-0.187470,-0.189512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.373527,0.641701,-0.189924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [308]:
pd.DataFrame(y_train).head()

,0
0,-0.797681
1,0.218468
2,-0.870263
3,0.581378
4,-1.305755


In [309]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4527, 9293)
(4527, 1)
(1132, 9293)
(1132, 1)


In [310]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators=100)# 100 trees
reg.fit(X_train, y_train)
pred = reg.predict(X_test)
from sklearn.metrics import r2_score
score = r2_score(y_test, pred)
print("The accuracy of our model is {}%".format(round(score, 2) *100))

C:\Users\usman\AppData\Local\Temp\ipykernel_6416\1194427304.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  reg.fit(X_train, y_train)


The accuracy of our model is 39.0%


In [313]:
xgb_r = xg.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 10)
  
# Fitting the model
xgb_r.fit(X_train, y_train)
  
# Predict the model
pred = xgb_r.predict(X_test)

from sklearn.metrics import r2_score
score = r2_score(y_test, pred)
print("The accuracy of our model is {}%".format(round(score, 2) *100))

The accuracy of our model is 37.0%


## **TASK 5**

In [420]:
dataset = pd.read_csv("../Downloads/archive (3)/creditcard.csv")
dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [409]:
dataset.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [410]:
normal, anomalous = dataset.iloc[:, -1].value_counts()
print('normal examples ',normal,'\nanomalous examples ',anomalous)
dataset.values.shape

normal examples  284315 
anomalous examples  492


(284807, 31)

In [411]:
X_train = dataset.loc[dataset['Class'] == 0] # take all the normal examples
X_train = X_train.sample(frac = 0.6).iloc[:, :-1] # take 60% of the normal examples

dataset_reduced = dataset.drop(X_train.index) # remove examples in X_train from dataset

X_train = X_train.values
X_train.shape

(170589, 30)

In [412]:
normal, anomalous = dataset_reduced.iloc[:, -1].value_counts()
print('normal examples ',normal,'\nanomalous examples ',anomalous)

normal examples  113726 
anomalous examples  492


In [413]:
X = dataset_reduced.iloc[:, :-1].values
y = dataset_reduced.iloc[:, -1].values
from sklearn.model_selection import train_test_split
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.4, stratify=y)
print(X_val.shape)
print(X_test.shape)
print(y_val.shape)
print(pd.DataFrame(y_val).value_counts())
print(y_test.shape)
print(pd.DataFrame(y_test).value_counts())

(68530, 30)
(45688, 30)
(68530,)
0    68235
1      295
dtype: int64
(45688,)
0    45491
1      197
dtype: int64


In [416]:
def estimate_gaussian(X): 
    """
    Calculates mean and variance of all features in the dataset
    """
    m, n = X.shape
    
    mu = 1 / m * np.sum(X, axis = 0)
    var = 1 / m * np.sum((X - mu) ** 2, axis = 0)
        
    return mu, var

def multivariate_gaussian(X, mu, var):
    
    k = len(mu)
    
    if var.ndim == 1:
        var = np.diag(var)
        
    X = X - mu
    p = (2* np.pi)**(-k/2) * np.linalg.det(var)**(-0.5) * \
        np.exp(-0.5 * np.sum(np.matmul(X, np.linalg.pinv(var)) * X, axis=1))
    
    return p

def select_threshold(y_val, p_val):
    best_epsilon = 0
    best_F1 = 0
    F1 = 0
    
    step_size = (max(p_val) - min(p_val)) / 1000000
    
    for epsilon in np.arange(min(p_val), max(p_val), step_size):
        predictions = (p_val < epsilon)
        tp = np.sum((predictions == 1) & (y_val == 1))
        fn = np.sum((predictions == 0) & (y_val == 1))
        fp = np.sum((predictions == 1) & (y_val == 0))
        prec = tp / (tp + fp)
        rec = tp / (tp + fn)
        F1 = 2 * prec * rec / (prec + rec)

        if F1 > best_F1:
            best_F1 = F1
            best_epsilon = epsilon
        
    return best_epsilon, best_F1

first we find mean and variance of each column/feature in train set

then we find probability of each example in CV using multivariate_gaussian and the mean and variance of each column/feature

an example is an anomaly if probability < epsilon

multiple values of epsilon are used to get predicted anomalies. The predicted and actual y of the CV is compaired to get F1 score. Epsilon with most F1 score is chosen

then we find probability of each example in train set using multivariate_gaussian and the mean and variance of each column/feature

each probability of train set is compaired with choosen epsilon

In [417]:
mu, var = estimate_gaussian(X_train)

# Evaluate the probabilites for the training set
probs = multivariate_gaussian(X_train, mu, var)

# Evaluate the probabilites for the cross validation set
probs_val = multivariate_gaussian(X_val, mu, var)

# Find the best threshold
epsilon, F1 = select_threshold(y_val, probs_val)

print('Best epsilon found using cross-validation: %e'% epsilon)
print('Best F1 on Cross Validation Set:  %f'% F1)
print('# Anomalies found: %d'% sum(probs < epsilon))

C:\Users\usman\AppData\Local\Temp\ipykernel_6416\266321648.py:37: RuntimeWarning: invalid value encountered in scalar divide
  prec = tp / (tp + fp)


Best epsilon found using cross-validation: 2.012659e-25
Best F1 on Cross Validation Set:  0.044250
# Anomalies found: 29376


In [419]:
probs_test = multivariate_gaussian(X_test, mu, var)
outliers = probs_test < epsilon
sum(outliers), sum(y_test)

(8067, 197)

In [431]:
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1:].values

In [432]:
pd.DataFrame(Y).head()

,0
0,0
1,0
2,0
3,0
4,0


In [433]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.transform(X_test)

In [434]:
pd.DataFrame(y_train).head()

,0
0,0
1,0
2,0
3,0
4,0


In [441]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm = pd.DataFrame(cm, columns = ['Pred 0', 'Pred 1'], index = ['Actual 0', 'Actual 1'])
cm

,Pred 0,Pred 1
Actual 0,56855,8
Actual 1,19,80


In [442]:
from sklearn.metrics import accuracy_score, auc, balanced_accuracy_score, confusion_matrix, f1_score, precision_score, average_precision_score, roc_auc_score,  recall_score,  precision_recall_curve
def print_scores(y_t, y_p):
    print(f'Accuracy  :{accuracy_score(y_t, y_p):.2f}' )
    print(f'Balanced  :{balanced_accuracy_score(y_t, y_p):.2f}' )
    print(f'F1        :{f1_score(y_t, y_p):.2f}' )
    print(f'Precision :{precision_score(y_t, y_p):.2f}' )
    print(f'Recall    :{recall_score(y_t, y_p):.2f}' )
    print(f'roc auc   :{roc_auc_score(y_t, y_p):.2f}' )
    print(f'pr auc    :{average_precision_score(y_t, y_p):.2f}' )
print_scores(y_test, y_pred)

Accuracy  :1.00
Balanced  :0.90
F1        :0.86
Precision :0.91
Recall    :0.81
roc auc   :0.90
pr auc    :0.73


In [437]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
from sklearn.model_selection import cross_val_score
acc = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv = 10) # 10 folds
print('{:.2f} %'.format(acc.mean()*100))
print('{:.2f} %'.format(acc.std()*100))

99.96 %
0.01 %
